In [1]:
from torch import empty
import math
from FC_module import LossMSE, ReLU, Tanh, Sigmoid, FCC, Sequential

In [ ]:
train_size = 1000
test_size = 200
valid_size = 100
train_input = empty(train_size, 2).uniform_(0,1)
train_target = train_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)

valid_input = empty(valid_size, 2).uniform_(0,1)
valid_target = valid_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)

test_input = empty(test_size, 2).uniform_(0,1)
test_target = test_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)

[Go to Benchamrking](#Benchmarking) with Pytorch

### These two are just for a quick check I know they are terrible :D

In [4]:
def accuracy_count(pred,true):
    pred = (pred > 0.5).long().view(-1)
    true = true.long().view(-1)
    return (pred.size(0) - (pred-true).abs().sum()).item()

In [ ]:
import torch
import matplotlib.pyplot as plt

Temporary!

In [ ]:
b = empty(10).uniform_(0,1).expand(5,10)
b[:,1]

In [ ]:
b = empty(5,100).fill_(25).mean(dim = 0)
b.shape

In [ ]:
keepdim 
empty(a.shape).shape

In [ ]:
len([5,7])

In [47]:
def modelTrainer(model, num_epoch,train_input,train_target,test_input,test_target, train_batch, test_batch):
    
    loss_track = []
    acc_track = []
    test_size = test_input.size(0)
    for epoch in range(num_epoch):
        minibatch = train_batch
        for i in range(0, train_input.size(0), minibatch):
            out,loss = model.train(train_input.narrow(0, i, minibatch), train_target.narrow(0, i, minibatch).unsqueeze(1))
        if epoch % (num_epoch/10) == 0:
            print("Loss:", loss.item())
        loss_track.append(loss)
        
        acc = 0
        count = 0
        minibatch = test_batch
        for i in range(0, test_size, minibatch):
            truth = test_target.narrow(0, i, minibatch)
            inp = test_input.narrow(0, i, minibatch)
            out = model.eval(inp)
            #print(out)
            acc += accuracy_count(out, truth)
        acc_track.append(acc/test_size)
    return model

In [48]:
def modelTester(model, test_input,test_target,test_batch):    
    acc = 0
    count = 0
    test_size = test_input.size(0)
    minibatch = test_batch
    for i in range(0, test_size, minibatch):
        truth = test_target.narrow(0, i, minibatch)
        inp = test_input.narrow(0, i, minibatch)
        out = model.eval(inp)
        #print(out)
        acc += accuracy_count(out, truth)

    return acc/test_size

In [54]:
#torch.set_grad_enabled(False)
train_size = 1000
test_size = 200
valid_size = 100
train_input = empty(train_size, 2).uniform_(0,1)
train_target = train_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)

valid_input = empty(valid_size, 2).uniform_(0,1)
valid_target = valid_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)

test_input = empty(test_size, 2).uniform_(0,1)
test_target = test_input.add(-0.5).pow(2).sum(1).sub(1 / (7)).multiply(-1).sign().add(1).div(2)


train_noise = empty(train_size, 2).normal_(0,0.1) 
valid_noise = empty(valid_size, 2).normal_(0,0.1) 
test_noise = empty(test_size, 2).normal_(0,0.1)

train_input += train_noise
valid_input += valid_noise
test_input += test_noise


seq = Sequential(["FCC","Tanh","FCC"],[[2,8],[], [8,1]],"MSE",momentum = 0.1)
trained_model  = modelTrainer(seq,1000, train_input,train_target,valid_input,valid_target, 20, 10)



print(modelTester(trained_model, test_input,test_target,10) )




Loss: 0.2296084463596344
Loss: 0.21399788558483124
Loss: 0.14162799715995789
Loss: 0.11288009583950043
Loss: 0.10970111936330795
Loss: 0.10947927087545395
Loss: 0.10960791260004044
Loss: 0.10969054698944092
Loss: 0.10970363765954971
Loss: 0.10966849327087402
0.88


### Benchmarking

In [ ]:
from torch import optim, Tensor, nn
from torch import Tensor
from torch import nn
import torch
import numpy as np
import matplotlib.pyplot as plt
torch.set_grad_enabled(True)

def create_model():
    return nn.Sequential(
        nn.Linear(2, 8),
        nn.ReLU(),
        nn.Linear(8, 1),
#         nn.ReLU(),
    )

def train_model(model, train_input, train_target):
    loss_track = []
    loss_func = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = 0.1)
    mini_batch = 200
    # Iterate through epochs
    for i in range(1000):
        #Iterate t hrough mini batches
        for b in range(0, train_input.size(0), mini_batch):
            # Forward pass selecting the corresponding minibacth
            output = model(train_input.narrow(0, b, mini_batch))
            # Calculate loss
            loss = loss_func(output.squeeze(), train_target.narrow(0, b, mini_batch))
#             print("Loss:", loss.item())
            # Reset zero grad
            model.zero_grad()
            # Backward pass
            loss.backward()
            #Update step
            optimizer.step()
        loss_track.append(loss)
    plt.plot(loss_track)
    
def compute_nb_errors(model, data_input, data_target):
        output = model(data_input)
        comp = output.argmax(dim = 1) != data_target.argmax(dim = 1)
        nb_errors = torch.where(comp)[0].size()[0]
        error_rate = nb_errors/data_input.size(0)
        print(f'There were a total of {nb_errors} errors, or {error_rate} of the testing set.')
        
        
model = create_model()
train_model(model, train_input, train_target)
# compute_nb_errors(model, test_input, test_target)